# Tradeoff between "error" and "effort" for a scalar system

Do all imports:

In [1]:
# Stuff for computation
import numpy as np
from scipy import integrate
from scipy import linalg

# Stuff for visualization
from ipywidgets import interactive_output, HBox, VBox, FloatSlider, Layout, Checkbox, FloatLogSlider
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import column, row, Spacer
from bokeh.plotting import figure
from bokeh.models import Div

[Display Bokeh plots inline](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html#classic-notebooks):

In [2]:
output_notebook()

Loading BokehJS ...

Suppress the use of scientific notation when printing small numbers:

In [3]:
np.set_printoptions(suppress=True)

This function returns the cost

$$ q x(t)^2 + r u(t)^2 $$

assuming that linear state feedback

$$ u = -k x $$

is applied to the (scalar) state-space system

$$ \dot{x} = ax + bu $$

starting from the initial condition

$$ x(t_0) = x_0. $$

In [4]:
def integrand(t, a, b, q, r, k, x0, t0):
    x = np.exp((a - b * k) * (t - t0)) * x0
    u = - k * x
    return q * x**2 + r * u**2

This function returns the total cost

$$ \int_{t_0}^{t_1} \left( q x(t)^2 + r u(t)^2 \right) dt $$

for the same controller, system, and initial condition.

In [5]:
def get_cost(a, b, q, r, k, x0, t0, t1):
    cost, err = integrate.quad(integrand, t0, t1, args=(a, b, q, r, k, x0, t0))
    return cost

This function returns the solution to the LQR problem

$$\begin{align*} \underset{u_{[t_0,\infty]}}{\text{minimize}} &\qquad\int_{t_0}^{\infty} \left( x(t)^T Q x(t) + u(t)^T R u(t) \right) dt\\ \text{subject to} &\qquad\dot{x}(t)=Ax(t)+Bu(t) \\ &\qquad x(t_0)=x_0. \end{align*}$$


In [6]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K, P

Create interactive visualization for the particular case in which $a=5$ and $b=1$ (there is nothing special about this case, it is just an example):

In [7]:
# Parameters that define the state-space model
(a, b) = (5., 1.)

# Parameters that define the initial conditions
x0 = 1.

# Parameters that define the cost
(q, r) = (1., 1.)

# Parameters that define the simulation
(t0, t1, dt) = (0., 3., 0.05)
nt = int(1 + np.ceil((t1 - t0) / dt))
t = np.linspace(t0, t1, nt)
k_list = np.linspace(5.001, 20, 50)

# Widgets
solve_lqr = Checkbox(description='Solve LQR')
show_costs = Checkbox(description='Show costs')
ks = FloatSlider(
    min=2,
    max=20,
    step=0.1,
    value=12,
    description='k',
    readout_format='.3f',
    layout=Layout(width='auto')
)
x0s = FloatSlider(
    min=-2,
    max=2,
    step=0.1,
    value=1,
    description='x0',
    readout_format='.3f',
    layout=Layout(width='auto')
)
qs = FloatLogSlider(min=-3, max=3, step=0.1, value=1, description='q', layout=Layout(width='auto'))
rs = FloatLogSlider(min=-3, max=3, step=0.1, value=1, description='r', layout=Layout(width='auto'))

# Function to update the figure after changing parameters
def update(k=12, x0=1, q=1, r=1, solve_lqr=False, show_costs=False):
    # Find optimal k (if desired)
    if solve_lqr:
        ks.disabled = True
        K, P = lqr(np.array([[a]]), np.array([[b]]), np.array([[q]]), np.array([[r]]))
        k = K[0, 0]
        ks.value = k
    else:
        ks.disabled = False
    
    # Get the closed-loop eigenvalue
    s = a - b * k
    
    # Get the state, input, and cost as functions of time
    x = np.exp((a - b * k) * (t - t0)) * x0
    u = - k * x
    cost = np.array([get_cost(a, b, q, r, k, x0, t0, t1) for t1 in t])

    # Get the cost at infinity
    if (a - b * k) < 0:
        cost_at_infinity = get_cost(a, b, q, r, k, x0, t0, np.inf)
    else:
        cost_at_infinity = np.inf
    
    # Get the cost at infinity as a function of k (if desired)
    c_list = np.zeros_like(k_list)
    if show_costs:
        for i, k_cur in enumerate(k_list):
            c_list[i] = get_cost(a, b, q, r, k_cur, x0, t0, np.inf)
    
    # Plot everything
    s_plt.data_source.data['x'] = [s]
    s_plt.data_source.data['y'] = [0]
    x_plt.data_source.data['y'] = x
    u_plt.data_source.data['y'] = u
    cost_plt.data_source.data['y'] = cost
    cost_at_infinity_plt.data_source.data['x'] = [k]
    cost_at_infinity_plt.data_source.data['y'] = [cost_at_infinity]
    costs_at_infinity_plt.data_source.data['x'] = k_list
    costs_at_infinity_plt.data_source.data['y'] = c_list
    
    # Refresh plots
    push_notebook()

# Plots
s_fig = figure(title='CLOSED-LOOP EIGENVALUES', height=300, width=300,
                x_range=(-15, 5), y_range=(-10, 10))
s_plt = s_fig.circle([0], [0], size=5, color='navy', alpha=0.5)
x_fig = figure(title='STATE', height=200, width=400,
                x_range=(t0, t1), y_range=(-2, 2))
x_plt = x_fig.line(t, np.zeros_like(t), line_width=2, line_color='navy')
u_fig = figure(title='INPUT', height=200, width=400,
                x_range=(t0, t1), y_range=(-15, 15))
u_plt = u_fig.line(t, np.zeros_like(t), line_width=2, line_color='navy')
cost_fig = figure(title='FINITE-HORIZON COST', height=200, width=400,
                x_range=(t0, t1), y_range=(0, 40))
cost_plt = cost_fig.line(t, np.zeros_like(t), line_width=2, line_color='navy')
cost_at_infinity_fig = figure(title='INFINITE-HORIZON COST', height=300, width=300,
                x_range=(0, 20), y_range=(0, 30))
costs_at_infinity_plt = cost_at_infinity_fig.line(
    k_list,
    np.zeros_like(k_list),
    line_width=2,
    line_color='cornflowerblue',
)
cost_at_infinity_plt = cost_at_infinity_fig.circle([0], [0], size=10, color='coral', alpha=1.0)

# Text
div_style = {'font-size': '150%'}
x_lab = Div(text=r'$$x(t) = e^{(a - bk)t}x_0$$', style=div_style, align='center')
u_lab = Div(text=r'$$u(t) = -kx(t)$$', style=div_style, align='center')
cost_lab = Div(text=r'$$\int_{0}^{t} (qx(t)^2 + ru(t)^2) dt$$', style=div_style, align='center')

# Layout (bokeh)
show(
    row(
        column(s_fig, cost_at_infinity_fig),
        column(Spacer(), sizing_mode='stretch_width'),
        column(row(x_fig, x_lab), row(u_fig, u_lab), row(cost_fig, cost_lab)),
    ),
    notebook_handle=True,
)

# Layout (widgets)
ui = HBox(
    [
        VBox([solve_lqr, show_costs], layout=Layout(width='20%')),
        VBox([ks, x0s], layout=Layout(width='30%')),
        VBox([qs, rs], layout=Layout(width='50%'))
    ], layout=Layout(border='solid 1px', width='100%')
)
out = interactive_output(
    update,
    {
        'k': ks,
        'x0': x0s,
        'q': qs,
        'r': rs,
        'solve_lqr': solve_lqr,
        'show_costs': show_costs,
    }
)
display(ui, out)

Output()